In [ ]:
import cohere
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()

# Verify that the environment variable is set
api_key = os.getenv("COHERE_API_KEY")
if not api_key:
    raise ValueError("COHERE_API_KEY environment variable is not set")

# Initialize Cohere client
co = cohere.Client(api_key)

# Sample documents
documents = [
    "Hybrid search combines dense and sparse methods for better retrieval.",
    "Dense retrieval search uses embeddings for semantic matching.",
    "Sparse retrieval uses keyword matching like TF-IDF or BM25.",
    "Cohere provides powerful NLP models for dense retrieval."
]

# Generate dense embeddings using Cohere
response = co.embed(texts=documents)
document_embeddings = response.embeddings

# Function to get TF-IDF vectors
def get_tfidf_vectors(documents):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    return tfidf_matrix, vectorizer

# Get TF-IDF vectors for documents
tfidf_matrix, vectorizer = get_tfidf_vectors(documents)

# Example query
query = "What is hybrid search?"

# Generate dense embedding for query using Cohere
query_embedding = co.embed(texts=[query]).embeddings[0]

# Get TF-IDF vector for query
query_tfidf_vector = vectorizer.transform([query])

# Sparse retrieval using cosine similarity on TF-IDF vectors
sparse_scores = cosine_similarity(query_tfidf_vector, tfidf_matrix).flatten()

# Dense retrieval using cosine similarity on dense embeddings
dense_scores = cosine_similarity([query_embedding], document_embeddings).flatten()

# Print documents after sparse (keyword) search
print("Documents ranked by keyword search (TF-IDF):")
for idx in np.argsort(sparse_scores)[::-1]:
    print(f"Score: {sparse_scores[idx]:.4f}, Document: {documents[idx]}")

print("\n")

# Print documents after dense (semantic) search
print("Documents ranked by semantic search (embeddings):")
for idx in np.argsort(dense_scores)[::-1]:
    print(f"Score: {dense_scores[idx]:.4f}, Document: {documents[idx]}")

print("\n")

# Combine scores (simple weighted sum approach)
alpha = 0.5  # Weight for sparse scores
beta = 0.5   # Weight for dense scores
combined_scores = alpha * sparse_scores + beta * dense_scores

# Rank documents based on combined scores
ranked_indices = np.argsort(combined_scores)[::-1]

# Print final ranked documents
print("Documents ranked by combined scores (hybrid search):")
for idx in ranked_indices:
    print(f"Score: {combined_scores[idx]:.4f}, Document: {documents[idx]}")
